# Imports

In [1]:
import pandas as pd
import re

# Read in the data

In [84]:
df = pd.read_csv('../data/raw/harvard/training/1/106886', sep="\t", header=None)
df.columns = ['words']
df

,words
0,RECORD #106886
1,192629344 | NH | 68247963 | | 3666218 | 7/10/2...
2,Discharge Date: 3/17/2006
3,"ATTENDING: HEISSER III , MONROE JOSEPH MD"
4,ADDENDUM
...,...
133,Dictation ID 8624693
134,"Addendum Created by BRANDT , III , EUGENE TERE..."
135,A: 5/22/06
136,Dictation ID : 2917664CEX


In [87]:
# df.loc[df.make == '', 'make'] = df.Words.str.split().str.get(0)
df['first_word'] = df['words'].str.split().str.get(0)
record = df.loc[df.first_word == 'RECORD', 'words'].str.split("#").str.get(1)
principal_row = df[df['first_word']=='PRINCIPAL'].index[0]
eScription_row = df[df['first_word']=='eScription'].index[0]
df_reduced = df.iloc[principal_row:eScription_row,]
df_reduced.head(20)


,words,first_word
6,PRINCIPAL DIAGNOSIS: Coronary artery disease.,PRINCIPAL
7,HISTORY OF PRESENT ILLNESS: Mr. Midget is a 56...,HISTORY
8,with a past medical history significant for in...,with
9,"infarction in 1995 , which was treated with an...",infarction
10,"time. Since then , he has been stable on medic...",time.
11,approximately two months prior to this admissi...,approximately
12,noting anterior chest pain with exertion. He u...,noting
13,"test , which was found to be abnormal , and wa...",test
14,catheterization. Cardiac catheterization was p...,catheterization.
15,10/3/06 at Au Hospital . This study revealed a,10/3/06


In [105]:
df["uppercase"] = df.first_word.apply(lambda x: x.isupper())
df["uppercase_count"] = df.uppercase.cumsum()
df["count"] = df.index
df.groupby('uppercase_count')['words'].apply(" ".join).iloc[16]

'DISCHARGE INSTRUCTIONS: The patient is instructed to make and keep all of his follow-up appointments and shower and wash his wounds with soap and water each day. He is to continue diuresis with Lasix 40 mg p.o. daily for five days and then discontinue until his re-evaluation at a follow-up appointment with his PCP or cardiologist and 1-2 weeks. Of note , this patient was diagnosed for the first time as a type II diabetic during this admission with a hemoglobin A1c of 7.2 prior to admission. He was followed by the Diabetes Management Service postoperatively , and was started on metformin 500 mg a day and should follow up with his primary care physician in 1-2 weeks for follow-up check of his blood sugar and any necessary adjustment in his diabetic medications. He is instructed to make follow-up appointments with his cardiac surgeon Dr. Jakowich for two weeks following discharge from the hospital and to return to see his primary care physician , Dr. Parmalee one to two weeks following d

In [88]:
# Extract Record number als index wert
for i, row in df.iterrows():
    if row.first_word.islower():
        print(row.words)

with a past medical history significant for inferior myocardial
infarction in 1995 , which was treated with angioplasty at that
time. Since then , he has been stable on medical management until
approximately two months prior to this admission , when he began
noting anterior chest pain with exertion. He underwent a stress
test , which was found to be abnormal , and was referred for cardiac
catheterization. Cardiac catheterization was performed on
artery , a 90% mid segment stenosis of his first diagonal coronary
artery , a 50% mid segment stenosis of his circumflex coronary
artery , a 90% ostial stenosis of his first obtuse marginal , a
flow was noted from the left anterior descending to the posterior
descending coronary artery. Given the results of this cardiac
catheterization , he was referred for elective revascularization.
significant for hypertension , hypercholesterolemia , lower back
pain , nephrolithiasis as well as a 3.4 cm infrarenal abdominal
aortic aneurysm at the bifurcatio

In [31]:
with open("../data/raw/harvard/training/1/155092",'r') as f:
    lines = f.readlines()
    important = False
    for line in lines:
        if line.startswith("PRINCIPAL") or important:
            print(line.split()[0] == "PRINCIPAL")
            print(line)
            important = True
            if line.startswith("eScription"):
                important = False
    # Principal Diagnosis
    # Zusammenführen der Zeilen bis zum nächsten großgeschriebenen 
    # Unnötiges am Ende entfernen
    

True
PRINCIPAL DIAGNOSIS: POST MYOCARDIAL INFARCTION ANGINA

False
SIGNIFICANT PROBLEMS: 1. Coronary artery disease

False
2. Hypertension

False
3. Non-insulin dependent diabetes

False
mellitus

False
HISTORY OF PRESENT ILLNESS: Patient is a 72 year old white male

False
Status post recent myocardial

False
infarction who was admitted for recurrent chest discomfort.

False
On the 1 of July , the patient noticed a "uncomfortable chest

False
sensation" as he was driving. He experienced diaphoresis at that

False
time , but no shortness of breath and no nausea or vomiting. He was

False
seen at an outside hospital and noted to have 2-3 mm ST elevations

False
in leads 2 , 3 , FV5 and V6 ( Humpde Hospital ). The patient was given

False
TPA , Heparin intravenous and aspirin. His peak CPK was noted to be

False
5 , 742 with an MB fraction of 199. The patient's hospital stay was

False
complicated by frequent PVCs on cardiac monitor , for which he was

False
temporarily treated with Lidoc

# Cleaning the data

## Format the data

In [5]:
df = df.loc[:, ["description", "medical_specialty", "sample_name", "transcription", "keywords"]]
df.head()

,description,medical_specialty,sample_name,transcription,keywords
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [93]:
df["medical_specialty"].value_counts()

 Surgery                          1103
 Consult - History and Phy.        516
 Cardiovascular / Pulmonary        372
 Orthopedic                        355
 Radiology                         273
 General Medicine                  259
 Gastroenterology                  230
 Neurology                         223
 SOAP / Chart / Progress Notes     166
 Obstetrics / Gynecology           160
 Urology                           158
 Discharge Summary                 108
 ENT - Otolaryngology               98
 Neurosurgery                       94
 Hematology - Oncology              90
 Ophthalmology                      83
 Nephrology                         81
 Emergency Room Reports             75
 Pediatrics - Neonatal              70
 Pain Management                    62
 Psychiatry / Psychology            53
 Office Notes                       51
 Podiatry                           47
 Dermatology                        29
 Cosmetic / Plastic Surgery         27
 Dentistry               

In [94]:
section_words = [
"SUBJECTIVE",
"MEDICATIONS",
"ALLERGIES",
"OBJECTIVE",
"ASSESSMENT",
"PLAN",]
def get_all_section_words(sentence):
    title_information_dict = {}
    for i in range(len(section_words)-1):
        start = section_words[i]
        end = section_words[i +1]
        print(start, end)
        
        # getting index of substrings
        try:
            start_index = sentence.index(start)
            end_index = sentence.index(end)
        except ValueError as e:
            print("ValueError", e)
            continue
        
        # length of substring 1 is added to
        # get string from next character
        res = sentence[start_index + len(start) + 1: end_index]
        
        title_information_dict[start] = res.strip(", ")
    
    return title_information_dict
    
    
df["section_dict"] = df.head(2)["transcription"].apply(lambda x: get_all_section_words(x))

SUBJECTIVE MEDICATIONS
This 23-year-old white female presents with complaint of allergies.  She used to have allergies when she lived in Seattle but she thinks they are worse here.  In the past, she has tried Claritin, and Zyrtec.  Both worked for short time but then seemed to lose effectiveness.  She has used Allegra also.  She used that last summer and she began using it again two weeks ago.  It does not appear to be working very well.  She has used over-the-counter sprays but no prescription nasal sprays.  She does have asthma but doest not require daily medication for this and does not think it is flaring up.
MEDICATIONS ALLERGIES
Her only medication currently is Ortho Tri-Cyclen and the Allegra.
ALLERGIES OBJECTIVE
She has no known medicine allergies.
OBJECTIVE ASSESSMENT
Vitals:  Weight was 130 pounds and blood pressure 124/78.,HEENT:  Her throat was mildly erythematous without exudate.  Nasal mucosa was erythematous and swollen.  Only clear drainage was seen.  TMs were clear.,Ne

In [91]:
df.head()

,description,medical_specialty,sample_name,transcription,keywords,section_dict
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller...","{'SUBJECTIVE': ', This 23-year-old white fema..."
1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh...","{'MEDICATIONS': ', None.,'}"
2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart...",NaN
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple...",NaN
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo...",NaN


In [92]:
df["section_dict"].apply(pd.Series)

,SUBJECTIVE,MEDICATIONS,ALLERGIES,OBJECTIVE,ASSESSMENT,0
0,", This 23-year-old white female presents with...",", Her only medication currently is Ortho Tri-...",", She has no known medicine allergies.,",",Vitals: Weight was 130 pounds and blood pres...",", Allergic rhinitis.,",NaN
1,NaN,", None.,",NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
4994,NaN,NaN,NaN,NaN,NaN,NaN
4995,NaN,NaN,NaN,NaN,NaN,NaN
4996,NaN,NaN,NaN,NaN,NaN,NaN
4997,NaN,NaN,NaN,NaN,NaN,NaN
